<a href="https://colab.research.google.com/github/hsarfraz/Tiny-Machine-Learning/blob/main/0_6_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook I will be talking about classification which is a supervised machine learning method where a model tries to predict the correct label/category of input data (images). In classification, a model is fully trained using training data and is then evaluated on test data to perform predictions on new unseen data.

## Re-cap: Dataset types in ML model training:
To re-cap from [notebook 0.4](https://github.com/hsarfraz/Tiny-Machine-Learning/blob/main/0_4_Graphing_Neural_Network_Accuracy.ipynb), here are definitions of the different types of datasets used during machine learning (ML) model training:

 Types of ML Datasets  | Definition
 ------------- | -------------
 Training Dataset  | The dataset that is initially used to fit/train the ML model
 Validation Dataset  | The dataset used during the training phase of the model and used to fine-tune the model's parameters and make it accurate  
 Test Dataset  | The dataset used after the ML model has been fully trained to assess the model's perfomance on completly new/unseen data

 *  **NOTE:** Using a validation dataset is optional when training a ML model, but it is highly recommended because it improves the models accuracy. Some experts say that a validation dataset is not required if the ML model has no hyperparameters (ML models that don't have tuning options)

## Overview of Artificial Intelligence, Machine Learning, and Deep Learning:

Classification is a supervised learning method. Supervised learning is a subcategory of artificial intelligence and machine learning. Before I start talking about classification in greater detail I wanted to include a diagram which highlights the difference between artificial intelligence, machine learning, and deep learning.

<img src="images/0.6_AI_ML_DL_Overview.jpg" width="700">


### Defining Supervised Learning, Classification, and Regression

Here is a diagram which provides a detailed explanation of supervised learning (I will define the other machine learning methods in future notebooks):

<img src="images/0.6_supervised_learning.jpg" width="700">

### Connection between Supervised Learning and Deep Learning

*  Neural Networks:
  *  Neural networks are used in deep learning algorithms and they train data by mimicking the connection of nodes in the human brain ([notebook 0.3](https://github.com/hsarfraz/Tiny-Machine-Learning/blob/main/0_3_Neural_Network_Basics.ipynb) includes a illustration of a neural net)
  *  To re-cap from [notebook 0.5](https://github.com/hsarfraz/Tiny-Machine-Learning/blob/main/0_5_weights_and_bias_in_neural_network.ipynb) each node takes in inputs, weights, a bias, and produces a output. If the output values exceeds the given threshold value (this is established by the activation function), then the node is activated or "fires" and passes the output to the next layer of the network.
  *  Neural Networks learn this process through supervised learning (the process that I am referring to is how each node uses greadient decent to minimize the loss function with the use of inputs, weights, and bias). When the loss functions result is at or near zero (it has a slope of zero) then we can be confident in the model's accuracy to produce a correct output/prediction of the input data.

### Difference of Classification and Regression Neural Network Architecture

I have included another image which highlights the difference between the neural network architecture of a classification ML model and Regression model. The methodology of having multiple neurons in the output layer is often used in classification and the having one neuron in the output layer is often used in regression.

<img src="images/0.6_classification_regression_NN_architecture.jpg" width="700">




# Explaining the Code

In [1]:
# Load the necessary libraries
import sys
import tensorflow as tf
import numpy

# Checking to see if TensorFlow 2 and Python 3 since this script requires TensorFlow 2 and Python 3.
if sys.version_info.major < 3:
    raise Exception((f"The script is developed and tested for Python 3. "
                     f"Current version: {sys.version_info.major}"))

if tf.__version__.split('.')[0] != '2':
    raise Exception((f"The script is developed and tested for tensorflow 2. "
                     f"Current version: {tf.__version__}"))

## Loading the dataset `MNIST` and normalizing the images (Feature Scaling)

The `MNIST` dataset is already built into Tensorflow. It is a dataset of images that are labelled from 0-9. The dataset is split into training images (60,000) and labels, as well as validation images (10,000) and labels that is used for testing/validation.

### What is Feature Scaling?

Feature scaling is a method used to normalise a range of independent variables or features in a dataset. Feature scaling is also known as data normalisation and is performed during the data preprocessing stage (which is an iterative process where raw data is transformed and cleaned into a usable format). For example, if you have two independent variables with different ranges (ex. salary and age) you would want to apply feature scaling to these variables so they can all be on the same range (such as being centred around 0 or from 0-1). The type of range in which you want to place the data depends on the scaling technique used. The two most common feature scaling techniques are **normalisation and standardisation** which I will define below:

* **Normalisation (aka min-max scaling or min-max normalization)**: It is the simplest feature scaling method and rescales the independent variable feature range to [0,1].
  * The general normalisation formula is given below (the **max(x)** and **min(x)** are the feature's maximum and minimum values):
  $$
  x'=\dfrac{x-min(x)}{max(x)-min(x)}
  $$
  * If you would like to would like to specify your interval to be in any [a,b] range then you can specify the range using the formula below:
  $$
  x'=a+\dfrac{(x-min(x))(b-a)}{max(x)-min(x)}
  $$
* **Standardisation (aka feature standardisation)**: Makes the data values, in each feature, have a mean of 0 and unit variance. The formula uses the features distributed mean and standard deviation and calculates the new data point with this formula ( $\overline{x}$ is the average of the feature vector and $\sigma$ is the standard deviation of the feature vector):

 $$
 x’ = \dfrac{x-\overline{x}}{\sigma}
 $$

### When to use normalisation or standardisation?

When building a machine learning pipeline it is important to consider whether you are going to normalise or standardise your dataset features. I have highlighted some situations when it is best to use each feature scaling method.

* **Normalisation**: Used when the data distribution does not follow the Gaussian distribution. It is also useful in algorithms that do not assume the data distribution like K-Nearest Neighbors (a classification algorithm which is under supervised learning). Another popular use case of normalisation is when an algorithm requires the data to be on a 0-1 scale. For example, in image processing it is essential to normalise the pixel intensity to fit within a certain range (the range of pixels is from 0-255 for the RGB colour range)

* **Standardisation**: Used when the data follows a Gaussian Distribution (although this is not always the case). Since standardisation does not have a bounding range any outlets in the data will not be affected by standardisation. Standardisation is useful in clustering analysis (an unsupervised learning algorithm), to compare the similarities found between features based on certain distance measures. Another use case is during principal components analysis when individuals are interested in the components that maximise the data variance (something which min-max scaling does not present)



In [2]:
### LOADING MNIST DATASET
data = tf.keras.datasets.mnist
(training_images, training_labels), (val_images, val_labels) = data.load_data()

print("NOTE: I am printing out the max. and min. range of each image to show why I am dividing each image pixel by 255 for normalisation")
print()

### GETTING THE MAX AND MIN PIXCEL VALUES WITHOUT USING NUMPY (SLICING METHOD)
print("\033[1mFinding the max and min pixel using slicing\033[0m")
print("Here is the minimum pixel value in the training image dataset: ", training_images[..., 1].min())
print("Here is the maximum pixel value in the training image dataset: ", training_images[..., 1].max())

print()
### GETTING THE MAX AND MIN PIXCEL VALUES USING NUMPY (NUMPY METHOD)
print("\033[1mFinding the max and min pixel using numpy\033[0m")
print("Here is the minimum pixel value in the training image dataset: ", numpy.amin(training_images))
print("Here is the minimum pixel value in the training image dataset: ", numpy.amax(training_images))

### FEATURE SCALING (NORMALIZATION)
training_images  = training_images / 255.0
val_images = val_images / 255.0

11490434/11490434 [==============================] - 2s 0us/step
NOTE: I am printing out the max. and min. range of each image to show why I am dividing each image pixel by 255 for normalisation

Finding the max and min pixel using slicing
Here is the minimum pixel value in the training image dataset:  0
Here is the maximum pixel value in the training image dataset:  255

Finding the max and min pixel using numpy
Here is the minimum pixel value in the training image dataset:  0
Here is the minimum pixel value in the training image dataset:  255


## Defining the Neural Network Model

The code below defines the neural network that will be used to categorize the MNIST images.

```python
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(20, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
```

*  `tf.keras.models.Sequential([])` defines a sequential neural network. A sequential model consists of a linear stack of layers which are added to the model in sequential order. Each layer's output is fed as input to the next layer.
*  Each layer of the neural network is defined in the square brackets.
  *  `tf.keras.layers.Flatten(input_shape=(28,28))` is an operation that flattens out the images so they can be fed into the neural network. The flattened layer is added because the images in the MNIST dataset are 28 by 28, but the next layer has 20 neurons (a dimension of 20 by 1) so the dimension of the image needs to be something by 1 in order to fit in the layer. Flattening the image transforms its dimension from 28 by 28 to 784 by 1. `input_shape=(28,28)` tells the flattened layer to only take in images that have an input shape of 28 by 28. If it gets any other image shape an error will occur.
  *  `tf.keras.layers.Dense(20, activation=tf.nn.relu)` is a dense layer which means that all neurons in this layer will get their source of input data from all the neurons in the previous layer. There are 20 neurons in this layer and it has the relu activation function. It is important to make sure that the correct number of neurons are added to the layer. If you assign too many neurons in the layer you can overspecialize or your neural network could be slow to learn. If the number of neurons is too small then your neural network won’t have enough neurons to learn the contents of the image (I will explore methodologies on how to pick the optimal number of neurons later)
  *  `tf.keras.layers.Dense(10, activation=tf.nn.softmax)` is the final dense layer which has 10 neurons because the MNIST dataset has 10 classes (the digits from 0 to 9). This layer has the softmax activation function

I have included a image below which illustrates the neural network architecture:

<img src="images/0.6_MNIST_neural_net_architecture.jpg" width="700">

## Defining the relu and softmax activation functions

* **ReLU (rectified linear unit) activation function**: Every neuron will call its activation function assigned to each layer. The ReLU activation function changes any neuron output that is less than 0 to 0. So when $y=mx+b$ is evaluated on any of the neurons, in the first layer, the values that are less than zero will be thrown away and will be converted to zero. The ReLU activation function is commonly used in dense layers so the neurons don’t cancel each other out. For example, if we did not have the ReLU activation function and one neuron returns a value of 10 and the other returns a value of -10 then both values will cancel each other out. The ReLU activation function establishes a non-linear relationship between the layers which can help capture complex relationships between the layers.
* **softmax activation function**: The softmax activation function is called in the last layer and will help find the neuron, from the 10, that has the highest value.

In [3]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(20, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#to hide epoch output add `verbose=0`
model.fit(training_images, training_labels, epochs=20, validation_data=(val_images, val_labels))


Epoch 1/20
1875/1875 [==============================] - 11s 4ms/step - loss: 0.4157 - accuracy: 0.8817 - val_loss: 0.2737 - val_accuracy: 0.9211
Epoch 2/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2511 - accuracy: 0.9284 - val_loss: 0.2269 - val_accuracy: 0.9344
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2111 - accuracy: 0.9398 - val_loss: 0.2061 - val_accuracy: 0.9426
Epoch 4/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1835 - accuracy: 0.9477 - val_loss: 0.1818 - val_accuracy: 0.9484
Epoch 5/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1643 - accuracy: 0.9528 - val_loss: 0.1730 - val_accuracy: 0.9505
Epoch 6/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1503 - accuracy: 0.9563 - val_loss: 0.1586 - val_accuracy: 0.9524
Epoch 7/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1390 - accuracy: 0.9597 - val_loss: 0.1623 - val_accuracy

## Examine the test data

Using model.evaluate, you can get metrics for a test set. In this case we only have a training set and a validation set, so we can try it out with the validation set. The accuracy will be slightly lower, at maybe 96%.  This is because the model hasn't previously seen this data and may not be fully generalized for all data. Still it's a pretty good score.

You can also predict images, and compare against their actual label. The [0] image in the set is a number 7, and here you can see that neuron 7 has a 9.9e-1 (99%+) probability, so it got it right!


In [13]:
model.evaluate(val_images, val_labels)

classifications = model.predict(val_images)

count = 0
for x in classifications[0]:
  count += 1
  print('probability of neuron', count, ':' , x)

#print(classifications[0])
print()
print('neuron',val_labels[0],'has the highest probability of',classifications[0][6])

313/313 [==============================] - 0s 1ms/step
probability of neuron 1 : 1.2636455e-09
probability of neuron 2 : 7.0626794e-12
probability of neuron 3 : 8.122745e-05
probability of neuron 4 : 0.0001044048
probability of neuron 5 : 4.7030657e-11
probability of neuron 6 : 1.1008105e-08
probability of neuron 7 : 9.04926e-15
probability of neuron 8 : 0.999806
probability of neuron 9 : 6.816234e-06
probability of neuron 10 : 1.4793163e-06

neuron 7 has the highest probability of 9.04926e-15


## Modify to inspect learned values

This code is identical, except that the layers are named prior to adding to the sequential. This allows us to inspect their learned parameters later.

In [14]:
data = tf.keras.datasets.mnist

(training_images, training_labels), (val_images, val_labels) = data.load_data()

training_images  = training_images / 255.0
val_images = val_images / 255.0
layer_1 = tf.keras.layers.Dense(20, activation=tf.nn.relu)
layer_2 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    layer_1,
                                    layer_2])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=20)

model.evaluate(val_images, val_labels)

classifications = model.predict(val_images)
print(classifications[0])
print(val_labels[0])

Epoch 1/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4066 - accuracy: 0.8877
Epoch 2/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2349 - accuracy: 0.9324
Epoch 3/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1989 - accuracy: 0.9431
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1759 - accuracy: 0.9495
Epoch 5/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1597 - accuracy: 0.9539
Epoch 6/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1477 - accuracy: 0.9572
Epoch 7/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1377 - accuracy: 0.9604
Epoch 8/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1292 - accuracy: 0.9626
Epoch 9/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1228 - accuracy: 0.9642
Epoch 10/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.116

# Inspect weights

If you print layer_1.get_weights(), you'll see a lot of data. Let's unpack it. First, there are 2 arrays in the result, so let's look at the first one. In particular let's look at its size.

In [22]:
print('size of the first array of the neural network (the first layer):',layer_1.get_weights()[0].size)

size of the first array of the neural network (the first layer): 15680


The above code should print 15680. Why?

Recall that there are 20 neurons in the first layer.

Recall also that the images are 28x28, which is 784.

If you multiply 784 x 20 you get 15680.

So...this layer has 20 neurons, and each neuron learns a W parameter for each pixel. So instead of y=Mx+c, we have
y=M1X1+M2X2+M3X3+....+M784X784+C in every neuron!

Every pixel has a weight in every neuron. Those weights are multiplied by the pixel value, summed up, and given a bias.


In [19]:
print('number of neurons in the first layer of the neural network:',layer_1.get_weights()[1].size)

number of neurons in the first layer of the neural network: 20


The above code will give you 20 -- the get_weights()[1] contains the biases for each of the 20 neurons in this layer.

## Inspecting layer 2

Now let's look at layer 2. Printing the get_weights will give us 2 lists, the first a list of weights for the 10 neurons, and the second a list of biases for the 10 neurons

Let's look first at the weights:

In [23]:
print('array size of the second layer in the neural network:',layer_2.get_weights()[0].size)

array size of the second layer in the neural network: 200


This should return 200. Again, consider why?

There are 10 neurons in this layer, but there are 20 neurons in the previous layer. So, each neuron in this layer will learn a weight for the incoming value from the previous layer. So, for example, the if the first neuron in this layer is N21, and the neurons output from the previous layers are N11-N120, then this neuron will have 20 weights (W1-W20) and it will calculate its output to be:

W1N11+W2N12+W3N13+...+W20N120+Bias

So each of these weights will be learned as will the bias, for every neuron.

Note that N11 refers to Layer 1 Neuron 1.


In [21]:
print('number of neurons in the second layer of the neural network:',layer_2.get_weights()[1].size)

number of neurons in the second layer of the neural network: 10


...and as expected there are 10 elements in this array, representing the 10 biases for the 10 neurons.

Hopefully this helps you see how the element of a simple neuron containing y=mx+c can be expanded greatly into a deep neural network, and that DNN can learn the parameters that match the 784 pixels of an image to their output!